In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import scipy
from metpy.units import units
import metpy.calc as mpcalc
import math
import datetime as dt

import sys
sys.path.append('/Users/clintonalden/Documents/Research/summa_work/')
from utils import lw_clr
from utils import forcing_filler as ff


In [17]:
# Read in csv data from mazama Pass SNOTEL
mazama = pd.read_csv('Mazama_2170_2023.csv')
mazama24 = pd.read_csv('Mazama_2170_2024.csv')

# Concatenate the two dataframes
mazama = pd.concat([mazama, mazama24], ignore_index=True)

# Sort the dataframe by the 'time' column in ascending order
mazama.sort_values('Date/Time (PST)', inplace=True)

# Load template forcing file with correct attributes
template = xr.open_dataset('../summa_forcing_template.nc')

# Pull in only necessary columns and rename to match SUMMA formatting
mazama = mazama[['Date/Time (PST)', 'Temperature  (deg F) ', 'Wind Speed Average  (mph) ', 'Solar Radiation  (W/m2) ',
                  'Precipitation  (") ', 'Relative Humidity  (%) ']]
replace = {'Temperature  (deg F) ':'airtemp','Wind Speed Average  (mph) ':'windspd', 'Solar Radiation  (W/m2) ':'SWRadAtm', 
           'Precipitation  (") ':'pptrate', 'Date/Time (PST)':'time', 'Relative Humidity  (%) ':'rh'}
mazama.rename(columns=replace, inplace=True)

# Convert 'time' column to datetime format
mazama['time'] = pd.to_datetime(mazama['time'])

# Convert time column to desired format
mazama['time'] = mazama['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Convert precip from in hr^-1 to m hr^-1
mazama['pptrate'] = mazama['pptrate']/39.37
# Convert precipitation rate from m hr^-1 to kg m^-2 s^-1
mazama['pptrate'] = mazama['pptrate']/3.6

# Convert temperature from F to K
mazama['airtemp'] = (mazama['airtemp']-32)*(5/9)+273.15

# Set NaN SW radiation values to 0
# mazama['SWRadAtm'] = mazama['SWRadAtm'].fillna(0)

# Pyranometer at Mazama Pass looks biased low by 3.81 W m-2, adding to fix
# mazama['SWRadAtm'] = mazama['SWRadAtm'] + 3.81

# Set NaN precip values to 0
# mazama['pptrate'] = mazama['pptrate'].fillna(0)

# Generate pressure from hypsometric equation and site elevation (1981m)
# ff.fill_pressure(mazama, 1981)

# Generate specific humidity
# ff.fill_spec_hum(mazama)

# Generate longwave radiation
# mazama['LWRadAtm'] = lw_clr.dilleyobrien1998(mazama['airtemp'], mazama['rh'])

# Drop unnecessary columns
# mazama = mazama.drop(columns=['accppt', 'rh'])

# Set time as index
# mazama.set_index('time', inplace=True)
# mazama.index = pd.to_datetime(mazama.index)

mazama



,time,airtemp,windspd,SWRadAtm,pptrate,rh
8759,2023-01-01 00:00:00,267.805556,0.000,0.000000,0.0,91.00
8758,2023-01-01 01:00:00,266.783333,0.000,0.003100,0.0,89.40
8757,2023-01-01 02:00:00,265.388889,0.000,0.000000,0.0,88.50
8756,2023-01-01 03:00:00,264.038889,0.000,0.009301,0.0,86.90
8755,2023-01-01 04:00:00,263.561111,0.000,0.000000,0.0,86.50
...,...,...,...,...,...,...
8764,2024-03-27 09:00:00,274.822222,0.028,316.728500,0.0,63.38
8763,2024-03-27 10:00:00,275.383333,1.285,514.916100,0.0,76.91
8762,2024-03-27 11:00:00,275.305556,2.095,567.806000,0.0,85.70
8761,2024-03-27 12:00:00,276.377778,1.739,793.194500,0.0,85.20


In [11]:
# Create timesteps
# Specify the start date and time
start_date = '2023-10-17'
start_time = '00:00'

# Combine start date and time to create a datetime object
start_datetime = pd.to_datetime(f'{start_date} {start_time}')

# Specify the number of rows
num_rows = 3607

# Generate datetime values for the specified number of rows
time = pd.date_range(start=start_datetime, periods=num_rows, freq='H')  # 'H' for hourly intervals

# Set index of dataframe to new timesteps
harts.set_index(time, inplace=True)
harts.index.name = 'time'

/var/folders/db/7c91t8d577n1zsyhth1xc7840000gn/T/ipykernel_42734/2125568859.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time = pd.date_range(start=start_datetime, periods=num_rows, freq='H')  # 'H' for hourly intervals


In [4]:
# Convert dataframe to xarray
dsx = harts.to_xarray()

# Loop through variables and add attributes from template forcing file
for data_var in dsx:
    dsx[data_var].attrs = template[data_var].attrs
    
# Add hru dimension
dsx = dsx.expand_dims(dim={'hru':1})

# Add gap-filled and datastep variables
dsx['gap_filled'] = xr.DataArray(np.ones((1,dsx.time.shape[0])),dims = ['hru','time'])
dsx['data_step'] = 3600 # 3600 seconds for 1hr timesteps

# Transpose gap filled variable to match dimensions with the rest
# dsx['gap_filled'] = dsx['gap_filled'].T

# Convert all to float64
for var in dsx.data_vars:
    dsx[var] = dsx[var].astype(np.float64)

# Set hruID based on template
dsx['hruId'] = (xr.DataArray(np.ones((1))*template['hruId'].values,dims = ['hru'])).astype(np.int32)

# Transpose all variables to match SUMMA dimensions
count = 0
for var in dsx.data_vars:
    print(var,count)
    count += 1
    if count <= 7:
        attribs = dsx[var].attrs
        arr_t = dsx[var].values.T
        dsx[var] = xr.DataArray(dims = ['time','hru'],data = arr_t)
        dsx[var].attrs = attribs

# Set encoding for the time variable
# dsx['time'].encoding = {'_FillValue': np.nan, 'units': 'hours since 1990-01-01', 'calendar': 'proleptic_gregorian'}

# Set hruID based on template
dsx['hruId'] = (xr.DataArray(np.ones((1))*template['hruId'].values,dims = ['hru'])).astype(np.float64).fillna(0).astype(np.int32)

# Save to netcdf
dsx.to_netcdf('./harts_SNTL_WY24.nc',
              engine='h5netcdf')

airtemp 0
windspd 1
SWRadAtm 2
pptrate 3
airpres 4
spechum 5
LWRadAtm 6
gap_filled 7
data_step 8
hruId 9


In [5]:
dsx


<xarray.Dataset> Size: 260kB
Dimensions:     (time: 3607, hru: 1)
Coordinates:
  * time        (time) datetime64[ns] 29kB 2023-10-17 ... 2024-03-15T12:00:00
Dimensions without coordinates: hru
Data variables:
    airtemp     (time, hru) float64 29kB 277.6 276.2 276.0 ... 276.3 275.8 276.0
    windspd     (time, hru) float64 29kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    SWRadAtm    (time, hru) float64 29kB 53.66 4.68 3.81 3.81 ... 3.81 3.81 3.81
    pptrate     (time, hru) float64 29kB 0.0 0.001411 0.0 0.0 ... 0.0 0.0 0.0
    airpres     (time, hru) float64 29kB 8.71e+04 8.703e+04 ... 8.702e+04
    spechum     (time, hru) float64 29kB 0.005594 0.005401 ... 0.004635 0.004653
    LWRadAtm    (time, hru) float64 29kB 255.0 250.2 248.8 ... 245.6 243.8 244.7
    gap_filled  (hru, time) float64 29kB 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    data_step   float64 8B 3.6e+03
    hruId       (hru) int32 4B 1001